# A tronbot using neat

In [1]:
import neat

In [2]:
from tronlib import *

In [3]:
# Create an evaluation function


from __future__ import print_function
import os
import neat
import visualize
import numpy as np

# If changing these, update config as well
BOARD_WIDTH = 10
BOARD_HEIGHT = 10



def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = evaluate_single_genome(genome, config)
            
def evaluate_single_genome(genome, config, n_games=3):
    score = 0
    # Play n games
    bot2 = NNBot(genome, config)
    for i in range(n_games):
        board = TronBoard(BOARD_WIDTH, BOARD_HEIGHT, (3, 4), (5, 4), (1, 0), (1, 0))
        bot1 = SimpleBot()
        game = TronGame(board, bot1, bot2, show=False)
        
        for step, state in enumerate(game):
            pass  # Game updated
        # Winning is a big deal
        if game.board.winner == 2:
            score += 10000
        
        # More score for lasting longer
        score += step
        
    return score

class NNBot:
    
    def __init__(self, genome, config):
        self.net = neat.nn.FeedForwardNetwork.create(genome, config)
    
    def decide_move(self, board, player_position):
        move_choice = np.argmax(self.net.activate(board.board.flatten()))
        return TronBoard.VALID_VECTORS[move_choice]

ImportError: No module named visualize

In [4]:
%%writefile tron.config

#--- parameters for the XOR-2 experiment ---#

[NEAT]
fitness_criterion     = max
fitness_threshold     = 500000
pop_size              = 150
reset_on_extinction   = False

[DefaultGenome]
# node activation options
activation_default      = sigmoid
activation_mutate_rate  = 0.0
activation_options      = sigmoid

# node aggregation options
aggregation_default     = sum
aggregation_mutate_rate = 0.0
aggregation_options     = sum

# node bias options
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 30.0
bias_min_value          = -30.0
bias_mutate_power       = 0.5
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1

# genome compatibility options
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient   = 0.5

# connection add/remove rates
conn_add_prob           = 0.5
conn_delete_prob        = 0.5

# connection enable options
enabled_default         = True
enabled_mutate_rate     = 0.01

feed_forward            = True
initial_connection      = full_direct

# node add/remove rates
node_add_prob           = 0.2
node_delete_prob        = 0.2

# network parameters
num_hidden              = 5
num_inputs              = 100
num_outputs             = 4

# node response options
response_init_mean      = 1.0
response_init_stdev     = 0.0
response_max_value      = 30.0
response_min_value      = -30.0
response_mutate_power   = 0.0
response_mutate_rate    = 0.0
response_replace_rate   = 0.0

# connection weight options
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_max_value        = 30
weight_min_value        = -30
weight_mutate_power     = 0.5
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1

[DefaultSpeciesSet]
compatibility_threshold = 3.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 20
species_elitism      = 2

[DefaultReproduction]
elitism            = 2
survival_threshold = 0.2


Overwriting tron.config


In [5]:

def run(config):

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 300)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    #winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    
    #node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
    #visualize.draw_net(config, winner, True, node_names=node_names)
    visualize.plot_stats(stats, ylog=False, view=True)
    visualize.plot_species(stats, view=True)

    p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
    p.run(eval_genomes, 50)
    return p, stats

In [ ]:
# Load configuration.
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     "tron.config")

p, stats = run(config)

# Parallel (Python 2 only!)

In [11]:
import pickle
import zlib

import pp
# list of available servers
#servers = ("server1:port1, "server2:port2", "server3:port3")
servers = () # empty: run local
# you can set the number of CPUs the local machine will use
# if you set ncpus = 0, then only distributed workers will
# receive and do the job
job_server = pp.Server(ncpus=4, ppservers=servers)  #, loglevel=20)
print("Starting pp with", job_server.get_ncpus(), "workers")



def parallel_evaluate(genomes, config):
    # number of chunks (jobs)
    num_chunks = 2
    # size for each subpopulation
    size = config.pop_size/num_chunks
    # make sure we have a proper number of chunks
    assert config.pop_size % num_chunks == 0, "Population size is not multiple of num_chunks"

    jobs = []
    for k in xrange(num_chunks):
        # divide the population in chunks and evaluate each chunk on a
        # different processor or machine
        print ('Chunk %d:  [%3d:%3d]' %(k, size*k, size*(k+1)))

        # compressing the population is useful when running
        # on a network of machines over the internet
        # It drastically reduces the lag at minimal cost
        # zlib compression (for small chromosomes)
        #           ratio             ratio
        # level   protocol=0 (note that protocol 1 or 2 are even better)
        #   1       74.06%
        #   3       77.21%
        #   6       79.68%
        #   9       80.58%

        # first pickles the population object
        pickle_pop = pickle.dumps((genomes[size*k:size*(k+1)], config), 2)
        # then compress the pickled object
        compressed_pop = zlib.compress(pickle_pop, 3)
        #print "Ratio: ", len(pickle_pop), len(compressed_pop)
        # submit the job
        jobs.append(job_server.submit(parallel_evaluation,
                                      args=(compressed_pop, k),
                                      depfuncs=(),
                                      modules=('neat','zlib','math')))

    all_jobs =[] # the results for all jobs
    for k in xrange(num_chunks):
        all_jobs += (jobs[k]())
    # assign the fitness back to each chromosome
    for i, fitness in enumerate(all_jobs):
        if not fitness:
            fitness = -1
        genomes[i][1].fitness = fitness


def parallel_evaluation(compressed_pop, chunk, n_games=20):
    # This function will run in parallel
    from tronlib import TronBoard, TronGame, SimpleBot, NNBot

    # don't print OS calls to stdout:
    #http://www.parallelpython.com/component/option,com_smf/Itemid,29/topic,103.0
    print("Evaluating chunk %d at %s" %(chunk, os.popen("hostname").read()))

    # decompress the pickled object
    decompress_pop = zlib.decompress(compressed_pop)
    # unpickle it
    sub_pop, config = pickle.loads(decompress_pop)

    # XOR-2
    INPUTS = ((0, 0), (0, 1), (1, 0), (1, 1))
    OUTPUTS = (0, 1, 1, 0)

    fitness = []
    for genome_num, genome in sub_pop:
        score = 0
        # Play n games
        bot2 = NNBot(genome, config)
        for i in range(n_games):
            board = TronBoard(10, 10, (3, 4), (5, 4), (1, 0), (1, 0))
            bot1 = SimpleBot()
            game = TronGame(board, bot1, bot2, show=False)

            step = 0
            while not game.board.gameover:
                game.update()
                step += 1
            # Winning is a big deal
            if game.board.winner == 2:
                score += 10000

            # More score for lasting longer
            score += step
        if score:
            fitness.append(score)
        else:
            fitness.append(-1)
    # when finished, return the list of fitness values
    return fitness
        

def run_parallel(config):

    
    
    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))

    p.run(parallel_evaluate, 50)
    #visualize.draw_ff(pop.stats[0][-1])
    job_server.print_stats()
    
    return p

Starting pp with 4 workers


In [12]:
# Load configuration.
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     "tron.config")

p = run_parallel(config)


 ****** Running generation 0 ****** 

Chunk 0:  [  0: 75]
Chunk 1:  [ 75:150]
Evaluating chunk 0 at ip-172-31-38-202

Evaluating chunk 1 at ip-172-31-38-202

Population's average fitness: 1093.83333 stdev: 3422.64824
Best fitness: 20136.00000 - size: (9, 920) - species 1 - id 69
Average adjusted fitness: 0.053
Mean genetic distance 2.589, standard deviation 0.303
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   150  20136.0    0.053     0
Total extinctions: 0
Generation time: 11.511 sec

 ****** Running generation 1 ****** 

Chunk 0:  [  0: 75]
Chunk 1:  [ 75:150]
Evaluating chunk 0 at ip-172-31-38-202

Evaluating chunk 1 at ip-172-31-38-202

Population's average fitness: 1845.88000 stdev: 5134.35328
Best fitness: 30115.00000 - size: (9, 916) - species 1 - id 183
Average adjusted fitness: 0.060
Mean genetic distance 2.493, standard deviation 0.351
Population of 150 members in 1 species:
   ID   ag

Evaluating chunk 1 at ip-172-31-38-202

Population's average fitness: 1932.06667 stdev: 4333.05245
Best fitness: 20173.00000 - size: (8, 763) - species 1 - id 1979
Average adjusted fitness: 0.096
Mean genetic distance 2.169, standard deviation 0.475
Population of 150 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   13    72  20173.0    0.104     4
     2    6    78  20101.0    0.088     3
Total extinctions: 0
Generation time: 9.157 sec (9.451 average)

 ****** Running generation 14 ****** 

Chunk 0:  [  0: 75]
Chunk 1:  [ 75:150]
Evaluating chunk 0 at ip-172-31-38-202

Evaluating chunk 1 at ip-172-31-38-202

Population's average fitness: 1854.98667 stdev: 4589.24333
Best fitness: 30076.00000 - size: (8, 765) - species 1 - id 2199
Average adjusted fitness: 0.060
Mean genetic distance 2.258, standard deviation 0.468
Population of 150 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======

Chunk 1:  [ 75:150]
Evaluating chunk 0 at ip-172-31-38-202

Evaluating chunk 1 at ip-172-31-38-202

Population's average fitness: 2803.32000 stdev: 5751.34054
Best fitness: 30201.00000 - size: (10, 541) - species 2 - id 3854
Average adjusted fitness: 0.090
Mean genetic distance 2.159, standard deviation 0.433
Population of 150 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   26    64  20199.0    0.077    17
     2   19    86  30201.0    0.102     4
Total extinctions: 0
Generation time: 7.730 sec (8.600 average)

 ****** Running generation 27 ****** 

Chunk 0:  [  0: 75]
Chunk 1:  [ 75:150]
Evaluating chunk 0 at ip-172-31-38-202

Evaluating chunk 1 at ip-172-31-38-202

Population's average fitness: 3207.60000 stdev: 6540.99363
Best fitness: 40187.00000 - size: (6, 520) - species 1 - id 4003
Average adjusted fitness: 0.082
Mean genetic distance 2.162, standard deviation 0.437
Population of 150 members in 2 species:
   ID  

Chunk 1:  [ 75:150]
Evaluating chunk 0 at ip-172-31-38-202

Evaluating chunk 1 at ip-172-31-38-202

Population's average fitness: 6835.22000 stdev: 10328.92141
Best fitness: 50242.00000 - size: (12, 516) - species 2 - id 5723
Average adjusted fitness: 0.131
Mean genetic distance 1.891, standard deviation 0.317
Population of 150 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   39    90  40263.0    0.157     2
     2   32    60  50242.0    0.105     0
Total extinctions: 0
Generation time: 7.038 sec (7.260 average)
Saving checkpoint to neat-checkpoint-39

 ****** Running generation 40 ****** 

Chunk 0:  [  0: 75]
Chunk 1:  [ 75:150]
Evaluating chunk 0 at ip-172-31-38-202

Evaluating chunk 1 at ip-172-31-38-202

Population's average fitness: 5644.43333 stdev: 8555.55604
Best fitness: 30284.00000 - size: (12, 517) - species 2 - id 5954
Average adjusted fitness: 0.178
Mean genetic distance 1.971, standard deviation 0.309
Popul

In [13]:
import random
from tronlib import *

def main():
    best_bot = NNBot(p.best_genome, config)

    board = TronBoard(10, 10, (3, 4), (5, 4), (1, 0), (1, 0))
    bot1 = SimpleBot()
    game = TronGame(board, bot1, best_bot, show=True)

    last_board = game.board.board

    def init():
        im.set_data(game.board.board)
        return (im,)


    def animate(i):
        global last_board
        if game.board.gameover:
            data = last_board
        else:
            game.update()
            data = game.board.board
        im.set_array(data)
        last_board = data
        return (im,)

    fig = plt.figure()
    x, y = game.board.board.shape
    #ax = plt.axes(xlim=(0, x), ylim=(0, y))
    #line, = ax.plot([], [], lw=2)
    im=plt.imshow(game.board.board,interpolation='nearest')

    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=100, interval=100, blit=True)

    return anim

In [17]:
pickle.dump(p.best_genome, open('best.genome', 'wb'))

In [18]:
anim = main()
from IPython.display import HTML
HTML(anim.to_html5_video())

RuntimeError: No MovieWriters available!